# Defining our own types

We have briefly seen a few examples of defining our own types in Julia. 

A type is like a template for a box, that contains data.
A simple example is a "volume" type, that represents the volume of an object:

In [ ]:
type Vol1
    value
end

In [ ]:
V = Vol1(3)

We can show this nicely by overloading the `show` function on our type:

In [ ]:
type Vol1a
    value
end

import Base: show

show(io::IO, V::Vol1a) = print(io, "Volume with value ", V.value)

[We have defined a new type, since in Julia 0.5 the redefinition of `show` will not have any effect, since the previous version has been cached.]

In [ ]:
V = Vol1a(3)

We can define e.g. the sum of two volumes:

In [ ]:
import Base: +

+(V1::Vol1a, V2::Vol1a) = Vol1a(V1.value + V2.value)

In [ ]:
V + V

But the following does not work, since we haven't defined `*` yet on our type:

In [ ]:
2V

**Exercise**: Define `*` of two `Vol`s  and of a `Vol` and a number.

There is a problem with our definition:

In [ ]:
Vol1("hello")

It doesn't make sense to have a string as a volume. So we should **restrict** which kinds of `value` are allowed, i.e. the **type** of `value`:

In [ ]:
type Vol2
    value::Float64
end

In [ ]:
Vol2(3.1)

In [ ]:
Vol2("hello")

# Different types of volume: **parameteric types**

Now we can imagine that in different contexts, we could want integer volumes, or rational volumes, rather than `Vol`s which contain a floating-point number, e.g. for a 3D printer that makes everything out of cubes of the same size. 

We could define the following sequence of different types.

In [ ]:
type Vol_Int
    value::Int
end

type Vol_Float
    value::Float64
end

type Vol_Rational
    value::Rational{Int64}
end

In [ ]:
Vol_Int(3)

In [ ]:
Vol_Int(3.1)

In [ ]:
Vol_Float(3.1)

But clearly this is the wrong way to do it, since we're repeating ourselves, and there is a strong principle not to do so (https://en.wikipedia.org/wiki/Don't_repeat_yourself).

Isn't there a more efficient way, where Julia itself can generate all of these different types?

What we would like to do is tell Julia that the **type** itself (here, `Int`, `Float64` or `Rational{Int64}`) 
is a **parameter** that we will specify. 

The syntax in Julia for this is to use curly braces (`{`, `}`) to specify such a **type parameter**:

In [ ]:
type Vol3{T}
    value
end

We can now pass in **any type** and `T` will be replaced by that type, creating a new type, e.g.

In [ ]:
V = Vol3{Float64}(3.1)

In [ ]:
typeof(V)

In [ ]:
V2 = Vol3{Int64}(4)

In [ ]:
typeof(V3)

We see that the types of `V1` and `V2` are *different* (but related), and we have achieved what 
we wanted.

The type `Vol3` is called a **parametric type**, with **type parameter** `T`. Parameteric types may have several type parameters, as we have already seen with `Array`s:

In [ ]:
a = [3, 4, 5]
typeof(a)

The type parameters here are `Int64`, which is itself a type, and the number `1`.

## Improving the solution

The problem with this solution is the following, which echos what happened at the start of the notebook:

In [ ]:
V = Vol3{Int64}(3.1)

In [ ]:
typeof(V.value)

The type of the element (here, `Float64`) is disconnected from the type parameter (`Int64`). 
So we have not yet actually captured the pattern of `Vol2`,
which restricted the `value` field to be of the desired type.

We solve this be specifying the field to **also be of type `T`**, with the **same `T`**:

In [ ]:
type Vol4{T}
    value::T
end

For example,

In [ ]:
V = Vol4{Int64}(3.0)

In [ ]:
typeof(V.value)

Now when we try to do 

In [ ]:
Vol4{Int64}(3.1)

Julia throws an error, namely an `InexactError()`.
This means that we are trying to "force" the number 3.1 into a "smaller" type `Int64`, i.e. one in which it can't be represented.

However, now we seem to be repeating ourselves again: We know that `3.1` is of type `Float64`, and in fact Julia knows this too; so it seems redundant to have to specify it. Can't Julia just work it out? Indeed it can!:

In [ ]:
Vol2(3.1)

Here, Julia has **inferred** the type `T` from the "inside out". That is, it did some pattern matching to realise that `value::T` was **matched** if `T` was chosen to be `Float64`, and then propagated this same value of `T` **upwards** to the type parameter.

## More fields

**Exercise**: Define a `Point` type that represents a point in 2D, with two fields. What are the options for this type, mirroring the types `Vol1` through `Vol4`?

## Summary

With parametric types, we have the following possibilities:

1. Julia converts (if possible) to the header type

2. Julia infers the header type from the inside (through the argument)


# Constructors

When we define a type, Julia also defines the **constructor functions** that we have been using above. These are functions with exactly the same name as the type.

They can be discovered using `methods`:

In [ ]:
methods(Vol1)

We see that Julia provides two default constructors.

For parametric types, it is a bit more complicated:

In [ ]:
methods(Vol4)

In [ ]:
methods(Vol4{Float64})

## Inner constructors

Julia allows us to provide our own constructor functions.
E.g.

In [ ]:
Vol2("3.1")

Here, we have tried to provide a numeric string, which is not allowed, since the string is not a number. We can add a constructor to allow this:

In [ ]:
Vol2(s::String) = Vol2(parse(Float64, s))

In [ ]:
Vol2("3.1")

In [ ]:
Vol2("hello")

We have added a new constructor outside the type definition, so it is called an **outer constructor**.

## Constructors that impose a restriction: **inner constructors**

Now consider the following:

In [ ]:
Vol2(-1)

Oops! A volume cannot be negative, but our constructors allow us to make a negative volume. To prevent this, we can define a constructor **within the type definition itself**, called **inner constructors**, that allows us to impose a restriction, or in general allows us to force objects to be constructed only in a certain way.

[In Julia, these are the **only methods** that may be defined inside the type definition. Unlike in object-oriented languages, methods **do not belong to types** in Julia; rather, they exist outside any particular type, and (multiple) dispatch is used instead.]

For example:

In [ ]:
type Vol5
    value::Float64
    
    function Vol5(V) 
        if V < 0
            throw(ArgumentError("Volumes cannot be negative"))
        end
        
        new(V)
    end
end

In [ ]:
Vol5(3)

In [ ]:
Vol5(-34)

If we define an inner constructor, then Julia no longer defines the standard constructors; this is why defining an inner constructor gives us exclusive control over how our objects are created.

## Inner constructors for parametric types

In [ ]:
type Vol6
    value::

# `type` vs `immutable`

So far we have been using `type`. But this allows the following:

In [ ]:
V = Vol5(3)

In [ ]:
V.value = -3

In [ ]:
V

Oops! We have violated the restriction on negative volumes.
To get around this, we use `immutable` instead of `type`:

In [ ]:
immutable Vol7
    value::Float64
    
    function Vol7(V) 
        if V < 0
            throw(ArgumentError("Volumes cannot be negative"))
        end
        
        new(V)
    end
end

In [ ]:
V = Vol7(-3)

In [ ]:
V = Vol7(3)

In [ ]:
V.value = -4

In [ ]:
V

## Performance gain with `immutable`s

The fact that an object is `immutable` also has performance implications, namely it allows the compiler to do optimizations to gain more performance.

An example is arrays: an `Array` of objects of an `immutable` type is stored directly in memory, whereas an `Array` of objets of a `type` are stored using pointers.

# Syntax changes in Julia 0.6

The syntax for parametric types will change in Julia 0.6.